In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import numpy as np
import pandas as pd

## 손글씨 숫자 인식

    - 이미 학습된 매개변수를 사용하여 학습 과정은 생략하고 순전파 신경망을 구현
    
### MNIST data set 구현 

    - 손글씨 숫자 이미 집합으로 널리 연구되는 데이터 셋.
    - 0 ~ 9 까지의 숫자 이미지로 훈련 이미지 60,000 장 ( Train ) , 시험 이미지 ( Test ) 10,000 장
    - 각 데이터는 28 X 28 크기의 회색조, 픽셀은 0~255 의 값을 갖음 
    - 인터넷에 다운받을 수 있다.
    
- URL

```python
url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}
```

In [7]:
# 첫번째 훈련 이미지가 무엇인지 나타내보자.

from mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train , t_train) , ( x_test , t_test ) = load_mnist(flatten=True , normalize=False )

img = x_train[0]
label = t_train[0]

label

5

In [8]:
print(img.shape)
img = img.reshape(28,28)
img_show(img) 

# 이미지 파일로  변환

(784,)


KeyboardInterrupt: 

## 신경망 추론 처리

    - 입력층(input) 뉴런을 784개 ( 28 X 28 ) , 출력층(최종층) 뉴런을 10개 ( 0~9 손글씨) 로 구성이 목적.
    
    - 은닉층(Layer) 은 총 2개로 1 번쨰 은닉층 은 50개의 뉴런 , 2번째 은닉층은 100개의 뉴런을 배치
    
    - 입력층 에서 첫 번째 은닉층으로 가는 가중치  크기 (W1) 는 ( 784 X 50)
    
    - 1번째 은닉층에서 -> 2번째 은닉층 가는 가중치 크기(W2)는 ( 50 X 100)
    
    - 2번째 은닉층 -> 출력층 가는 가중치(W3) 는 ( 100 X 10)
    
    - 출력층의 activation 함수는 softmax , 그 이외의 활성화 함수는 sigmoid 함수로 가정

In [5]:
import pickle
from functions import sigmoid , softmax

def get_data():   # 시험데이터 불러오는 함수 
    (x_train , t_train) , ( x_test , t_test ) = load_mnist(flatten=True , normalize=True , one_hot_label=False )
    return x_test , t_test

# pickle 을 이용해 학습된 가중치 매개변수 로드 

def init_network():
    with open('C:\\Users\\Hyobin An\\workspace-AI_Intro\\sample_weight.pkl' , 'rb') as f:
        network = pickle.load(f)
    return network


# Feed - forward NN  
def predict(network , x): 
    
    W1 , W2 , W3 = network['W1'] , network['W2'] , network['W3']
    b1 , b2 , b3 = network['b1'] , network['b2'] , network['b3']
    
    a1 = np.dot(x,W1)+b1  # 첫번째 output 
    z1 = sigmoid(a1)
    
    a2 = np.dot(z1,W2)+b2 # 두번쨰 output
    z2 = sigmoid(a2)
    
    a3 = np.dot(z2,W3)+b3 # 최종층 output 
    
    y = softmax(a3)
    
    return y 

x , t = get_data() # test data 가져오기
network = init_network() # 훈련된 가중치 가져오기
accuracy_cnt = 0 # 횟수 

# 미니배치
batch_size = 100 # 임의 결정 
curacy_cnt = 0 # ? 

# for i in range(len(x)):
    
#     y = predict(network , x[i])
#     p = np.argmax(y)
    
#     if p == t[1]: # p 가 Target 벨류와 같으면  횟수 +1 
#         accuracy_cnt += 1 
        
for i in range( len(x) , batch_size ):
    
    x_batch = x[i:i+batch_size]
    y_batch = predict(network , x_batch)
    
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum( p == t[i:i+batch_size])
    
print( " Accuracy = " + str( float(accuracy_cnt)/len(x)) )

 Accuracy = 0.0


## 미니배치 / 배치 처리
    
    - 전체적으로 784 개의 feature 로 구성된 1차원 배열이 입력되어 , 마지막에서는 feature 가 10개인 1차원 배열이 출력되는 흐름
    
    - 즉 , 이미지 데이터 1장만 입력하면 하나의 손글씨 숫자의 결과가 출력
    
    - 이미지를 여러 장 한꺼번에 입력하는 경우는?? 
    - 예를들어 이미지 100개를 묶어 predict() 함수에 한번에 입력하게 되면 
    - 100 X 784 개의 원소로 구성된 1차원 배열 ( vector ) 로 입력 형상이 바뀐다고 생각 할 수 있다.
    
    - 이처럼 하나로 묶은 데이터를 배치(batch) 라고 함, 즉, 배치가 묶음이란 의미로 이미지가 지폐처럼 다발로 묶여 있다고 생각하면 됨